In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorchModel

session = sagemaker.Session()
role = sagemaker.get_execution_role()

model_data_path = "s3://ivadas-data-722396408893/models/patchcore-mvtec-multiclass/patchcore_model.tar.gz"

patchcore_model = PyTorchModel(
    model_data=model_data_path,
    role=role,
    entry_point="inference_patchcore.py",
    source_dir="src",
    framework_version="2.1",
    py_version="py310",
    env={
        "SAGEMAKER_REQUIREMENTS": "requirements.txt",
        "MODEL_SERVER_WORKERS": "1",   #only one worker
    },
)

patchcore_predictor = patchcore_model.deploy(
    endpoint_name="ivadas-patchcore-mvtec",
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge",    #GPU instance with 16 GiB RAM
)

print("Deployed endpoint:", patchcore_predictor.endpoint_name)

----------!Deployed endpoint: ivadas-patchcore-mvtec


In [19]:
import base64
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

patchcore_predictor.serializer = JSONSerializer()
patchcore_predictor.deserializer = JSONDeserializer()

# pick any mvtec test image that exists in your Studio filesystem
test_img_path = "mvtec/hazelnut/test/good/000.png"  # adjust path

with open(test_img_path, "rb") as f:
    img_bytes = f.read()

payload = {
    "class_name": "hazelnut",                         # no "mvtec_" prefix here
    "image_base64": base64.b64encode(img_bytes).decode("utf-8"),
}

response = patchcore_predictor.predict(payload)
print(response)


{'class_name': 'hazelnut', 'anomaly_score': 2.7141027450561523}


In [3]:
!pwd

/home/sagemaker-user/patchcore-inspection


In [ ]:
good_img = "mvtec/hazelnut/test/good/000.png"
bad_img  = "mvtec/hazelnut/test/crack/000.png"   #adjust defect type if needed

def remote_score(img_path, cls="hazelnut"):
    with open(img_path, "rb") as f:
        img_bytes = f.read()
    payload = {
        "class_name": cls,
        "image_base64": base64.b64encode(img_bytes).decode("utf-8"),
    }
    return patchcore_predictor.predict(payload)

print("GOOD via endpoint:", remote_score(good_img))
print("DEFECT via endpoint:", remote_score(bad_img))


GOOD via endpoint: {'class_name': 'hazelnut', 'anomaly_score': 2.7141027450561523}
DEFECT via endpoint: {'class_name': 'hazelnut', 'anomaly_score': 6.399173736572266}
